## Installation

## Импорт библиотек

In [ ]:
import os
import re
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms.v2 as v2
import torchvision.transforms as T
import PIL
import numpy as np
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from tqdm import tqdm
import pandas as pd
from PIL import Image
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
from torch.nn.functional import softmax
import random
import cv2
from torchvision import models


In [ ]:
# from pytorch_grad_cam import GradCAM, GradCAMPlusPlus
# from pytorch_grad_cam.utils.model_targets import ClassifierOutputSoftmaxTarget
# from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

# Set seed
seed_everything(42)

## Важные переменные

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class1_folder = '/content/drive/MyDrive/plants_classification/P. multifida'
class2_folder = '/content/drive/MyDrive/plants_classification/P. turczaninovii'
hybrid_class_folder = '/content/drive/MyDrive/plants_classification/Hybrid'
project_folder = '/content/drive/MyDrive/plants_classification'

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_save_path = "/content/drive/MyDrive/plants_classification/models/best_model_resnet50_no_shuffle_ext_500ep.pth"

## Работа с данными

### Функции

In [ ]:
# Load an image and preprocess it
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (224, 224))  # Resize for MobileNetV2
    img = transforms.ToTensor()(img)
    img = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img)
    img = img.unsqueeze(0)  # Add batch dimension
    return img

# Function to get the class label
def get_class_label(preds):
    _, class_index = torch.max(preds, 1)
    return class_index.item()

def get_conv_layer(model, conv_layer_name):
    for name, layer in model.named_modules():
        if name == conv_layer_name:
            return layer
    raise ValueError(f"Layer '{conv_layer_name}' not found in the model.")

# Function to generate Grad-CAM heatmap
def compute_gradcam(model, img_tensor, class_index, conv_layer_name="layer4"):
    conv_layer = get_conv_layer(model, conv_layer_name)

    # Forward hook to store activations
    activations = None
    def forward_hook(module, input, output):
        nonlocal activations
        activations = output

    hook = conv_layer.register_forward_hook(forward_hook)

    # Compute gradients
    img_tensor.requires_grad_(True)
    preds = model(img_tensor)
    loss = preds[:, class_index]
    model.zero_grad()
    loss.backward()

    # Get gradients
    grads = img_tensor.grad.cpu().numpy()
    pooled_grads = np.mean(grads, axis=(0, 2, 3))

    # Remove the hook
    hook.remove()

    activations = activations.detach().cpu().numpy()[0]
    for i in range(pooled_grads.shape[0]):
        activations[i, ...] *= pooled_grads[i]

    heatmap = np.mean(activations, axis=0)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap

# Overlay heatmap on image
def overlay_heatmap(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(img, alpha, heatmap, 1 - alpha, 0)
    return superimposed_img

def get_grad_cam(img_path, out_name):
  global model
  # Example Usage
  img_tensor = preprocess_image(img_path)

  # Get model predictions
  with torch.no_grad():
      preds = model(img_tensor)
  class_index = get_class_label(preds)
  preds = softmax(preds)
  print(f"Вероятность принадлежности экземпляра к P. multifida: {preds[0][0]:.3f}")
  print(f"Вероятность принадлежности экземпляра к P. turczaninovii: {preds[0][1]:.3f}")
  print(f"Predicted Class Index: {class_index}")

  # Compute Grad-CAM heatmap
  heatmap = compute_gradcam(model, img_tensor, class_index)


  # Overlay heatmap on the original image
  output_img = overlay_heatmap(img_path, heatmap)

  # Save the heatmap
  cv2.imwrite(out_name, output_img)

### Преобразование меток

In [ ]:
annot_df = pd.read_csv('/content/drive/MyDrive/plants_classification/annotations.csv', index_col=0)

In [ ]:
annot_df['target'] = annot_df['target'].map({'P. multifida': 0, 'P. turczaninovii': 1})

In [ ]:
test_imgs = annot_df.loc[annot_df['mode'] == 'test', ['name', 'target']]
test_multifida = test_imgs[test_imgs.target == 0].name.values
test_turczaninovii = test_imgs[test_imgs.target == 1].name.values

### Модель

In [ ]:
base_img_preprocessing = v2.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
model = torchvision.models.resnet50(weights="DEFAULT")
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True
model.fc = nn.Linear(model.fc.in_features, 2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_save_path, map_location=device), strict=False)
model.to(device)
model.eval();

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 73.8MB/s]


In [ ]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Свертки

In [ ]:
list(model.layer1.named_children())

[('0',
  Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )),
 ('1',
  Bottleneck(
    (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_s

In [ ]:
layers = [model.layer1, model.layer2, model.layer3, model.layer4]
chosen_layers = []
for i, layer in enumerate(layers):
  for j in range(len(list(layer.named_children()))):
    name = f'Layer {i+1} Bottleneck {j} '
    chosen_layers.append({'name': name + 'Conv1',
                           'layer': layer[j].conv1})
    chosen_layers.append({'name': name + 'Conv2',
                           'layer': layer[j].conv2})
    chosen_layers.append({'name': name + 'Conv3',
                           'layer': layer[j].conv3})

In [ ]:
image_path = test_multifida[3] # Замените на путь к вашему изображению
image = Image.open(image_path)
input_batch = preprocess_image(image_path)
layer_activations = {}  # Словарь для хранения активаций
visual_map = {64: (8, 8),
              128: (16, 8),
              256: (16, 16),
              512: (32, 16),
              1024: (32, 32),
              2048: (64, 32)}

In [ ]:


def get_activation(name):
    def hook(model, input, output):
        layer_activations[name] = output.detach()
    return hook

# def visualize_activations(activations, layer_name):
#     # Визуализация карт признаков (feature maps) для данного слоя
#     num_filters = activations.shape[1]
#     nxdim, nydim = visual_map[num_filters]
#     fig, axes = plt.subplots(nxdim, nydim)
#     for i in range(nxdim):
#       for j in range(nydim):
#         ax = axes[i, j]
#         ax.imshow(activations[0, i * nydim + j].cpu().numpy(), cmap='gray') # Отображение как grayscale image
#         ax.axis('off') # Отключаем оси
#     fig.suptitle(layer_name)
#     fig.subplots_adjust(wspace=0.1, hspace=0.1)
#     return fig

for pair in chosen_layers:
  pair['layer'].register_forward_hook(get_activation(pair['name']))

with torch.no_grad():
    output = model(input_batch)

# for pair in chosen_layers:
#     activations = layer_activations[pair['name']]
#     fig = visualize_activations(activations, pair['name'])
#     plt.savefig(f'/content/drive/MyDrive/plants_classification/cnn_shots/multifida_{pair["name"]}.jpg')
#     plt.close(fig)

# # 7. Создание кадров для GIF:
# frames = []

# # Кадр 1: Исходное изображение
# fig_original, ax_original = plt.subplots()
# ax_original.imshow(image)
# ax_original.axis('off')
# plt.savefig('frame_1.jpg')
# plt.close(fig_original)

In [ ]:
import os

for (name, image_path) in
for pair in chosen_layers:
    activations = layer_activations[pair['name']]
    path = "/content/drive/MyDrive/plants_classification/cnn_gif_folders/multifida/data/" + f'{pair["name"]} filter_{1}.jpg'
    num_filters = activations.shape[1]
    fig, ax = plt.subplots()  # Создаем фигуру и оси один раз
    ax.imshow(activations[0, 0].cpu().numpy(), cmap='gray')
    ax.axis('off')
    ax.set_title(pair['name'] + f' Filter {1}')
    fig.savefig(path)  # Сохраняем фигуру
    ax.clear() # Очищаем оси для следующего фильтра
    plt.close(fig)




In [ ]:
fig, ax = plt.subplots()  # Создаем фигуру и оси один раз
ax.imshow(image)
ax.axis('off')
ax.set_title('Source image. P. multifida')
fig.savefig(path)  # Сохраняем фигуру
ax.clear() # Очищаем оси для следующего фильтра
plt.close(fig)

In [ ]:
path = "/content/drive/MyDrive/plants_classification/cnn_gif_folders/multifida/data/" + f'source.jpg'


### Grad-Cam

In [ ]:
# for i in range(5):
#   get_grad_cam(test_multifida[i], f'/content/drive/MyDrive/plants_classification/grad_cam_results/multifida_{i}.jpg')
#   get_grad_cam(test_turczaninovii[i], f'/content/drive/MyDrive/plants_classification/grad_cam_results/turczaninovii_{i}.jpg')

In [ ]:


# # ----------------------------------------------------------------------
# # 3. Define Transformations
# # ----------------------------------------------------------------------

# # Define image transformations, including resizing
# image_size = (224, 224)  # Grad-CAM often works best with a fixed size

# transform = transforms.Compose([
#     transforms.Resize(image_size),  # Resize the image
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# targets = [ClassifierOutputSoftmaxTarget(1)]
# # fix the target layer (after which we'd like to generate the CAM)
# target_layers = [model.layer4]
# cam = GradCAM(model=model, target_layers=target_layers) # Ensure CUDA is used if available



# print(input_tensor.size())
# # generate CAM
# grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
# cam_image = show_cam_on_image(np.float32(img)/255, grayscale_cams[0, :], use_rgb=True) # Convert PIL Image to numpy array and normalize
# cam_image = np.uint8(255 * cam_image)  # Convert back to uint8

# cam = np.uint8(255*grayscale_cams[0, :])
# cam = cv2.merge([cam, cam, cam])

# # Convert PIL Image to numpy array
# img_np = np.array(img)

# # Resize cam_image to match the dimensions of img_np
# cam_image_resized = cv2.resize(cam_image, (img_np.shape[1], img_np.shape[0])) # (width, height)

# # display the original image & the associated CAM
# images = np.hstack((img_np, cam_image_resized))
# final_image = PIL.Image.fromarray(images)
# final_image.show() # Or save it